# Music EDA

## connecting to Spotify API with spotipy

In [64]:
import os
import sys
import json
from json.decoder import JSONDecodeError
import webbrowser
import spotipy
import spotipy.util as util

In [ ]:
# User ID: stevay?si=yNi65NanRZeIRXlu0Gjobw

In [65]:
username = sys.argv[1]
username

'-f'

In [66]:
try: 
    token = util.prompt_for_user_token(username,
                                       scope=None,
                                       client_id = '39cc01d4b4a544ebad4e813e7190e606',
                                       client_secret = '21e5aa26f17247bcae6d6aeba252a49f',
                                       redirect_uri='http://google.com/')
except:
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username,
                                       scope=None,
                                       client_id = '39cc01d4b4a544ebad4e813e7190e606',
                                       client_secret = '21e5aa26f17247bcae6d6aeba252a49f',
                                       redirect_uri='http://google.com/')
    

# create spotify object
spotify_object = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=39cc01d4b4a544ebad4e813e7190e606&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com%2F in your browser


Enter the URL you were redirected to: https://www.google.com/?code=AQCWj939X9F0CGEi6LD2tDLp_aBDZmo847GbZ5czgVTobrJfWIAZ_Whnvk5ds43EeiqlLS_t1fQa1_LKC4xShqVJAHwVsMzhKkDAMHxFCls6pgya_VR0tDf-ViBhvdV_r_zm_xUfS7wbnpoClrWPI94uwtT33rLJ4d_SXWNUt2c9p3Oh8YawE29M




## pulling user metadata from spotify_object

In [26]:
user = spotify_object.current_user()

In [ ]:
# useful code to print JSON from spotify
# print(json.dumps(VARIABLE, sort_keys=True,indent=4))

In [28]:
print(json.dumps(user, sort_keys=True,indent=4))

{
    "display_name": "Steve Au-Yeung",
    "external_urls": {
        "spotify": "https://open.spotify.com/user/stevay"
    },
    "followers": {
        "href": null,
        "total": 9
    },
    "href": "https://api.spotify.com/v1/users/stevay",
    "id": "stevay",
    "images": [
        {
            "height": null,
            "url": "https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=10201893448121480&height=200&width=200&ext=1546041432&hash=AeSrqWrvfgHV9EEF",
            "width": null
        }
    ],
    "type": "user",
    "uri": "spotify:user:stevay"
}


## defining variables
- display_name
- followers total

In [30]:
display_name = user['display_name']
followers = user['followers']['total']

In [33]:
print(display_name)
print(followers)

Steve Au-Yeung
9


In [36]:
search_results = spotify_object.search('Radiohead',limit=1,type='artist') # limit to one item per artist name search

In [43]:
print(json.dumps(search_results, sort_keys=True,indent=4))

{
    "artists": {
        "href": "https://api.spotify.com/v1/search?query=Radiohead&type=artist&market=US&offset=0&limit=1",
        "items": [
            {
                "external_urls": {
                    "spotify": "https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb"
                },
                "followers": {
                    "href": null,
                    "total": 3670167
                },
                "genres": [
                    "alternative rock",
                    "art rock",
                    "melancholia",
                    "modern rock",
                    "permanent wave",
                    "rock"
                ],
                "href": "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb",
                "id": "4Z8W4fKeB5YxbusRsdQVPb",
                "images": [
                    {
                        "height": 640,
                        "url": "https://i.scdn.co/image/afcd616e1ef2d2786f47b3b4a8a6aeea24a72adc",
  

## artist details

In [53]:
artist = search_results['artists']['items'][0]
artist

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
 'followers': {'href': None, 'total': 3670167},
 'genres': ['alternative rock',
  'art rock',
  'melancholia',
  'modern rock',
  'permanent wave',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
 'id': '4Z8W4fKeB5YxbusRsdQVPb',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/afcd616e1ef2d2786f47b3b4a8a6aeea24a72adc',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/563754af10b3d9f9f62a3458e699f58c4a02870f',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/4067ea225d8b42fa6951857d3af27dd07d60f3c6',
   'width': 160}],
 'name': 'Radiohead',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}

In [54]:
artist_name = artist['name']
artist_id = artist['id']
artist_followers = artist['followers']['total']
artist_genres = artist['genres']

In [61]:
# open an image found under 'images'
webbrowser.open(artist['images'][0]['url'])

True

In [ ]:
# album and track details
track_uri = []
track_art = []
z = 0

In [67]:
# extract album data
album_results = spotify_object.artist_albums(artist_id,album_type='album')

In [60]:
print(json.dumps(album_results, sort_keys=True,indent=4))

{
    "href": "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums?offset=0&limit=20&include_groups=album",
    "items": [
        {
            "album_group": "album",
            "album_type": "album",
            "artists": [
                {
                    "external_urls": {
                        "spotify": "https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb"
                    },
                    "href": "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb",
                    "id": "4Z8W4fKeB5YxbusRsdQVPb",
                    "name": "Radiohead",
                    "type": "artist",
                    "uri": "spotify:artist:4Z8W4fKeB5YxbusRsdQVPb"
                }
            ],
            "available_markets": [
                "AD",
                "AE",
                "AR",
                "AT",
                "AU",
                "BE",
                "BG",
                "BH",
                "BO",
                "BR",
           

### Here’s a quick example of using Spotipy to list the names of all the albums released by the artist ‘Birdy’:

In [68]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
#spotify = spotipy.Spotify() # spotify_object

results = spotify_object.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify_object.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Beautiful Lies
Fire Within
Live In London
Birdy
Birdy
Birdy (Deluxe Version)


### Here’s another example showing how to get 30 second samples and cover art for the top 10 tracks for Led Zeppelin:

In [70]:
lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

#spotify = spotipy.Spotify()
results = spotify_object.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print ('track    : ' + track['name'])
    print ('audio    : ' + track['preview_url'])
    print ('cover art: ' + track['album']['images'][0]['url'])
    print ()

track    : Stairway To Heaven
audio    : https://p.scdn.co/mp3-preview/8226164717312bc411f8635580562d67e191a754?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/557a6058e3de72bf37ffcd2c12dd5932276df344

track    : Immigrant Song
audio    : https://p.scdn.co/mp3-preview/8455599677a13017978dcd3f4b210937f0a16bcb?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/e6e513e68c4eefdfc7bc213b35061c9c30b42dfa

track    : Whole Lotta Love
audio    : https://p.scdn.co/mp3-preview/ce11b19a4d2de9976d7626df0717d0073863909c?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/5fa3a6cc1848ea743a293d2088046746d1b09608

track    : Black Dog
audio    : https://p.scdn.co/mp3-preview/9b76619fd9d563a48d38cc90ca00c3008327b52e?cid=39cc01d4b4a544ebad4e813e7190e606
cover art: https://i.scdn.co/image/557a6058e3de72bf37ffcd2c12dd5932276df344

track    : Good Times Bad Times
audio    : https://p.scdn.co/mp3-preview/c1f024eb57b569b926c8e68cab0a6056dc7d96

### Finally, here’s an example that will get the URL for an artist image given the artist’s name:

In [72]:
import spotipy
import sys

#spotify = spotipy.Spotify()

if len(sys.argv) > 1:
    name = ' '.join(sys.argv[1:])
else:
    name = 'Radiohead'

results = spotify_object.search(q='artist:' + name, type='artist')
items = results['artists']['items']
if len(items) > 0:
    artist = items[0]
    print (artist['name'], artist['images'][0]['url'])